# inventories.ipynb
Author: UFO Software, LLC<br>
Created: Sunday, February 14, 2021 at 13:15<br>

License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html> <br>
This is free software: you are free to change and redistribute it.<br>
There is NO WARRANTY, to the extent permitted by law.<br>
<br>
Reads in the Inventories_0.csv file from the [December 2020 WA State Traceability Data](https://lcb.app.box.com/s/fnku9nr22dhx04f6o646xv6ad6fswfy9?page=1)<br>
The file has two partial columns that casue a Pandas error; this notebook compensates for that by only reading the valid columns bys using the usecols parameter of the read_csv function.<br>
If you plan on using this data with Dask you need to run this notebook and export the dataframe to a .parquet file.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

import missingno as msno

# change to point to the data's location
file_path = Path('../dec-2020')
pd.set_option('display.max_columns', None)

In [2]:
# The Inventories_0.csv file has two parital columns that pandas errors out on.  To fix this just read in the valid columns using the usecols paramter of the read_csv function.
cols = ['global_id',
        'created_at',
        'updated_at',
        'mme_id',
        'user_id',
        'external_id',
        'area_id',
        'batch_id',
        'lab_result_id',
        'lab_retest_id',
        'is_initial_inventory',
        #depreciated 'inventory_created_at',
        #depreciated 'inventory_packaged_at',
        #depreciated 'created_by_mme_id',
        'qty',
        'uom',
        'strain_id',
        'inventory_type_id',
        #depreciated 'additives',
        #depreciated 'serving_num',
        'sent_for_testing',
        'deleted_at',
        'medically_compliant',
        'legacy_id',
        'lab_results_attested',
        'lab_results_date',
        'global_original_id']

# parse the columns containing dates
date_cols = ['created_at',
             'updated_at',
             'deleted_at',
             'lab_results_date']

# specify data types to reduce memory footprint
col_dtypes = {'global_id':'string',
              'mme_id' : 'string',
              'user_id' : 'string',
              'external_id' : 'string',
              'area_id': 'string',
              'batch_id': 'string',
              'lab_result_id' : 'string',
              'lab_retest_id' : 'string',
              'is_initial_inventory' : 'bool',
              #'qty' : 'float16',  if you are not going to save to parquet you can use float16 otherwise use float32
              'qty' : 'float32',
              'uom': 'category',
              'strain_id' : 'string',
              'inventory_type_id' : 'string',
              'sent_for_testing' : 'bool',
              #'medically_compliant' : 'float16',  if you are not going to save to parquet you can use float16 otherwise use float32
              'medically_compliant' : 'float32', 
              'legacy_id' : 'string',
              'lab_results_attested' : 'string', # this should be bool but there are bad entries
              'global_original_id' : 'string'
             }

inventories_df = pd.read_csv(file_path / 'Inventories_0.csv', sep = '\t', encoding = 'utf-16', usecols = cols, parse_dates = date_cols, dtype=col_dtypes)
# all the datasets in the WA data use global_id but it has different meaning for each dataset which makes the data difficult to understand and causes issues with Pandas when trying to perform operations on more than one dataframe.
inventories_df.rename(columns={'global_id':'inventory_id'}, inplace=True)
inventories_df.head()

,inventory_id,created_at,updated_at,mme_id,user_id,external_id,area_id,batch_id,lab_result_id,lab_retest_id,is_initial_inventory,qty,uom,strain_id,inventory_type_id,sent_for_testing,deleted_at,medically_compliant,legacy_id,lab_results_attested,lab_results_date,global_original_id
0,WAJ412598.IN1,2018-01-31 17:24:32,2018-02-28 01:13:28,WAWA1.MMDJ,WAWA1.USAM,<NA>,WAJ412598.AR3,WAJ412598.BA1,<NA>,<NA>,False,0.0,ea,WAJ412598.STX,WAJ412598.TY1S,False,NaT,NaN,<NA>,0,NaN,<NA>
1,WAJ412598.IN2,2018-01-31 17:24:32,2018-01-31 17:24:32,WAWA1.MMDJ,WAWA1.USAM,<NA>,WAJ412598.AR3,WAJ412598.BA2,<NA>,<NA>,False,84.0,ea,WAJ412598.STX,WAJ412598.TY1S,False,NaT,NaN,<NA>,0,NaN,<NA>
2,WAJ412598.IN3,2018-01-31 17:24:30,2018-02-28 09:37:34,WAWA1.MMDJ,WAWA1.USAM,<NA>,WAJ412598.AR3,WAJ412598.BA3,<NA>,<NA>,False,0.0,ea,WAJ412598.STX,WAJ412598.TY1S,False,NaT,NaN,<NA>,0,NaN,<NA>
3,WAJ412598.IN4,2018-01-31 17:24:35,2018-02-28 01:14:14,WAWA1.MMDJ,WAWA1.USAM,<NA>,WAJ412598.AR3,WAJ412598.BA4,<NA>,<NA>,False,0.0,ea,WAJ412598.STX,WAJ412598.TY1S,False,NaT,NaN,<NA>,0,NaN,<NA>
4,WAJ412598.IN5,2018-01-31 17:24:32,2018-02-28 01:15:30,WAWA1.MMDJ,WAWA1.USAM,<NA>,WAJ412598.AR3,WAJ412598.BA5,<NA>,<NA>,False,0.0,ea,WAJ412598.STX,WAJ412598.TY1S,False,NaT,NaN,<NA>,0,NaN,<NA>


In [3]:
inventories_df.tail()

,inventory_id,created_at,updated_at,mme_id,user_id,external_id,area_id,batch_id,lab_result_id,lab_retest_id,is_initial_inventory,qty,uom,strain_id,inventory_type_id,sent_for_testing,deleted_at,medically_compliant,legacy_id,lab_results_attested,lab_results_date,global_original_id
48201542,WAJ412598.INSPHGW,2021-01-06 15:29:52,2021-01-06 15:29:52,WAWA1.MMDJ,WAWA1.USAM,<NA>,WAJ412598.AR2,WAJ412598.BALTE7B,<NA>,<NA>,False,24.0,ea,WAJ412598.STO07B,WAJ412598.TYMDKJK,False,NaT,NaN,<NA>,0,NaN,WAJ412598.INRLRHK
48201543,WAJ413180.INSPHGX,2021-01-06 15:29:53,2021-01-06 15:29:53,WAWA1.MMV2,WAWA1.US2BN,<NA>,WAJ413180.AR28OK,WAJ413180.BAM2XGS,<NA>,<NA>,False,50.0,gm,WAJ413180.STHUCF,WAJ413180.TYMDKJL,False,NaT,NaN,<NA>,0,NaN,WAJ413180.INS0XV7
48201544,WAJ412500.INSPHGY,2021-01-06 15:29:57,2021-01-06 15:29:57,WAWA1.MMRO,WAWA1.US6K1,<NA>,WAJ412500.AR1N9U,WAJ412500.BALFFSE,<NA>,<NA>,False,22.0,ea,WAJ412500.STIBF,WAJ413287.TYJAO,False,NaT,NaN,<NA>,0,NaN,WAJ412500.INQWCV4
48201545,WAJ412500.INSPHGZ,2021-01-06 15:29:58,2021-01-06 15:29:58,WAWA1.MMRO,WAWA1.US6K1,<NA>,WAJ412500.AR1N9U,WAJ412500.BALFFSI,<NA>,<NA>,False,10.0,gm,WAJ412500.STIFT,WAJ413287.TY22F5D,False,NaT,NaN,<NA>,0,NaN,WAJ412500.INQWCVV
48201546,WAJ412500.INSPHH0,2021-01-06 15:29:58,2021-01-06 15:29:58,WAWA1.MMRO,WAWA1.US6K1,<NA>,WAJ412500.AR1N9U,WAJ412500.BALFFST,<NA>,<NA>,False,43.0,gm,WAJ412500.STIFZ,WAJ416853.TYBR,False,NaT,NaN,<NA>,0,NaN,WAJ412500.INQWCWW


In [4]:
inventories_df.lab_results_attested.unique()

<StringArray>
['0', '1', <NA>, 'WAR421777.IN8UV0V']
Length: 4, dtype: string

Since this record is initial inventory the value for lab_results_attested should probably be 1 instead of WAR421777.IN8UV0V

In [5]:
inventories_df[inventories_df.lab_results_attested=='WAR421777.IN8UV0V']

,inventory_id,created_at,updated_at,mme_id,user_id,external_id,area_id,batch_id,lab_result_id,lab_retest_id,is_initial_inventory,qty,uom,strain_id,inventory_type_id,sent_for_testing,deleted_at,medically_compliant,legacy_id,lab_results_attested,lab_results_date,global_original_id
14865443,WAR421777.IN8UV2G,2019-01-22 07:38:36,2019-01-22 07:40:06,WAWA1.MM5Q,WAWA1.US2HA,WAJ413287.IN8P5MG,WAR421777.AR27MH,WAR421777.BA2PZIN,WAATTESTE.LRVME4,<NA>,True,1.0,ea,<NA>,WAR421777.TY9HCVG,False,NaT,0.0,<NA>,WAR421777.IN8UV0V,1,<NA>


In [6]:
inventories_df.loc[inventories_df.lab_results_attested=='WAR421777.IN8UV0V', 'lab_results_attested'] = '1'

Uncomment the line in the cell below to save the dataframe to a .parquet file to use with Dask

In [7]:
#inventories_df.to_parquet(path = file_path / 'Inventories_0.parquet', compression = 'None')